# MDS=Multidimensional Scaling

## 多次元構造の次元削減と可視化
個々のinherent structure は3N次元の配置空間内の点であり、それらの類似度から、近い構造同士を結びつけてネットワークを作っても、画面に表示することも直感にたよることもできない。何らかの関係性を見付けだすか、より低い次元に落とさなければ、人間は理解できない。

高次元のデータを低次元に射影することも、多変量データを少数のパラメータで表現するので、一種のモデリングと言える。

座標はわからないが、相対距離の大小関係だけがわかるような点の集合がある。これらを、与えられた次元の空間に、 相対位置関係を保ちながら点を埋めこむことを考える。

例えば、平面上の4つの点の相対距離だけがわかっているとしよう。この関係を再現するように点を3次元空間に配置すると、並進と回転の自由度を除けば点の場所は一意に決まり、点は平面上に位置する。つまり、うめこまれた空間の次元に関係なく、点は本来の次元を回復する。もし埋めこまれる空間のほうが、次元が小さい場合には、相対位置関係を完全に保ったまま埋めこむことはできず、歪みを生じる。次元を小さくしていきながら、歪みの大きさを監視すれば、点の配置に内在する実効的な空間の次元の大きさがわかるとともに、それを近似的に低次元で表示することができる。このような手法を多次元尺度法(multi-dimensional scaling, MDS)と呼ぶ。

### アルゴリズム(多次元尺度法)

相対位置情報$\delta_{ij}$を維持するような、低次元(これを埋め込み空間と呼ぶ)の点$x_i$の配置を考える。

1. 埋め込み空間に、ランダムに点$x_i$を配置する。
2. 点$x_i$と$x_j$の間の距離が$\delta_{ij}$に近づくように、ちょっとだけ点$x_i$と$x_j$を移動する。
3. すべての点の対$(i,j)$について2の作業を繰り返す。

## 実装例1

アメリカの州都の間の距離をもとに、州都の位置を予測してみる。

states.txtにはすべての州都の経度緯度情報があるので、これをそのままプロットすればもちろんそのまま地図を作れる。

In [ ]:
# ちょっと読みこみが面倒臭い

#states is a dict of dicts
states = dict()

for line in open("states.txt").readlines():
    
    # 1文字目が#の場合は無視
    if line[0] == "#":
        continue 

    cols = line.split(": ")
    
    # 2カラムない場合も無視
    if len(cols) < 2:
        continue
    
    if cols[0] == "Name":
        state = cols[1]
        states[state] = dict()
    else:
        states[state][cols[0]] = cols[1]
    
print(states)

In [ ]:
import numpy as np

# 州の名前と、経緯度だけのシンプルな表にする
statename = []
statepos  = []
for state in states:
    statename.append(state[:-1])
    statepos.append((float(states[state]["Capital Longitude"]), float(states[state]["Capital Latitude"])))

statepos = np.array(statepos)

In [ ]:
# とりあえずプロットしてみる
from matplotlib import pyplot as plt

plt.gca().set_aspect('equal', adjustable='box')
plt.scatter(statepos[:,0], statepos[:,1])

アラスカとハワイが遠く離れていることがわかる。これらの地図上での距離を表にする。

In [ ]:
from scipy.spatial.distance import cdist

# cdist()関数は座標から相対距離の表を生成する。
table = cdist(statepos, statepos, metric="euclidean")
table

manifold(多様体学習)の手法の中にMDSが含まれている。

In [ ]:
from sklearn import manifold

# マシンを初期化する
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=60, metric=True)

# データを与え、MDSを実行する
estim = mds.fit_transform(table)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.gca().set_aspect('equal', adjustable='box')
# 得られた点雲をプロットする。
plt.scatter(estim[:,0], estim[:,1])

相対座標しか与えていないので、全体の向きが変わったり、左右反転することがある。しかし、全体の形や大きさは保っているのがわかる。

この例の場合、予測すべき座標の数(50x2)に比べて、与えられた距離情報(50x49/2)のほうが圧倒的に多いので、非常に精密に位置を予測できる。

In [ ]:
# 回転と並進で重ねる

from math import sin,cos,pi

# posの重心を求めておく。
com = [np.average(statepos[:,0]), np.average(statepos[:,1])]

theta = 77*pi/180
rotmat = np.array([[cos(theta),-sin(theta)],[sin(theta),cos(theta)]])

# estimのほうを回転する
aligned = estim @ rotmat + com

plt.gca().set_aspect('equal', adjustable='box')
# 得られた点雲をプロットする。
plt.scatter(aligned[:,0], aligned[:,1])
plt.scatter(statepos[:,0], statepos[:,1])


試しに、相対距離tableの要素の1/3を消してしまうと?

In [ ]:
# 自作のMDS。自分で書いてもこれぐらいの長さで書ける。

def mds(p,d, nloop=300):

    # 点の数
    N = p.shape[0]

    E = 0.0
    # 繰り返し
    for k in range(nloop):
        
        # 異なる2つの粒子について
        for i in range(N):
            for j in range(i+1,N):

                # 未定義値以外なら、
                if d[i,j] != 0:
                    
                    # 相対位置
                    dp = p[j] - p[i]
                    # 相対距離
                    L  = (dp@dp)**0.5
                    
                    # 本来の距離との差
                    delta = L - d[i,j]
                    E += delta
                    
                    # 相対距離が遠すぎる場合は縮め、近すぎる場合は遠ざける。
                    p[j] -= delta*0.3*dp/L
                    p[i] += delta*0.3*dp/L
    # 誤差を返す
    return E

# tableと同じshapeで内容が0〜1の乱数が入ったarrayを生成する。
noise = np.random.random(table.shape)
# 対称化する
noise = (noise + noise.T)/2

# tableのうち、noiseの値が1/2よりも小さい要素を0 (未定義値)にする。
thanos = table.copy()
thanos[noise<1/2] = 0

# 初期配置
estim2 = np.random.random(statepos.shape)
# 自作MDS
E = mds(estim2, thanos)
print("Error:",E)

plt.gca().set_aspect('equal', adjustable='box')
# 得られた点雲をプロットする。
plt.scatter(estim2[:,0], estim2[:,1])

相対距離のデータの1/2が失われても、だいたい形は同じになっているようだ。

ずれを調べるために、特異値分解を使って回転行列を求める。

In [ ]:
# 重心を原点に
com_e2 = np.mean(estim2, axis=0)
estim2 -= com_e2

# もとデータのほうも重心を0にする
pos = statepos - np.mean(statepos, axis=0)

# 特異値分解 http://nghiaho.com/?page_id=671
H = pos.T @ estim2
u,s,vh = np.linalg.svd(H)
# 回転行列が得られる
R = vh.T@u

plt.gca().set_aspect('equal', adjustable='box')
# 得られた点雲をプロットする。
plt.scatter((estim2@R)[:,0], (estim2@R)[:,1])
plt.scatter(pos[:,0], pos[:,1])

思いきって7割を消してみる。さすがに形を維持できなくなってくる。

In [ ]:
# tableのうち、noiseの値が7/10よりも小さい要素を0 (未定義値)にする。
thanos = table.copy()
thanos[noise<7/10] = 0

# 初期配置
estim2 = np.random.random(statepos.shape)
# 自作MDS
E = mds(estim2, thanos)
print("Error:",E)

# 重心を原点に
com_e2 = np.mean(estim2, axis=0)
estim2 -= com_e2

# もとデータのほうも重心を0にする
pos = statepos - np.mean(statepos, axis=0)

# 特異値分解 http://nghiaho.com/?page_id=671
H = pos.T @ estim2
u,s,vh = np.linalg.svd(H)
# 回転行列が得られる
R = vh.T@u

plt.gca().set_aspect('equal', adjustable='box')
# 得られた点雲をプロットする。
plt.scatter((estim2@R)[:,0], (estim2@R)[:,1])
plt.scatter(pos[:,0], pos[:,1])

## 実装例2: 地下鉄料金表から路線図を推定する
[ここ](https://subway.osakametro.co.jp/guide/fare/fare/price.php)に大阪メトロの料金表がある。

https://subway.osakametro.co.jp/guide/libray/kusuuhyou/kusuuhyou20180401.pdf

地下鉄料金は駅間距離によって1区から5区に分類されている。この情報には、路線がどうつながっているかという情報は含まれていないが、各駅の間の距離は料金から逆に推定できる。

いま、この料金表しか情報がない時に、地下鉄の路線図(あるいは大阪の地図)を復元できるだろうか。

地下鉄路線図の場合には、埋め込み次元(鉄道が走っている地表)は2次元であることが事前にわかっているが、一般にはわからない場合もある。路線図を例えば一次元に間違ってうめこんでしまった場合、どうやってもまともな解は得られない。(津山線ならなんとかなる)

また、この問題の場合、手元にあるのは駅間の距離ではなく、区数の表だけなので、どれだけがんばっても厳密に$\delta_{ij}$を再現するような布置$\{x\}$は見付からない。

そこで、相対距離関係そのものを再現する代わりに、相対距離の*順位*を再現するように布置しよう、という考え方が生まれた。距離(計量)に依存しないので、非計量多次元尺度法(NMDS)と呼ばれる。

こちらも、手順はMDSの場合とさほど変わらない。

### アルゴリズム(非計量多次元尺度法)

相対位置に関する順位情報$\delta_{ij}$ (近いものほど小さい数になる)を維持するような、低次元(これを埋め込み空間と呼ぶ)の点$x_i$の配置を考える。

1. 埋め込み空間に、ランダムに点$x_i$を配置する。
2. 点$x_i$と$x_j$の間の距離の順位関係が$\delta_{ij}$に近づくように、ちょっとだけ点$x_i$と$x_j$を移動する。
3. すべての点の対$(i,j)$について2の作業を繰り返す。


In [ ]:
import numpy as np

table = np.loadtxt("blocks.txt")
stations = open("stations.txt").readlines()
stations

表の形を整形する(上下反転し、対称化する)

In [ ]:
table = table[::-1].copy()
table += table.T
table

In [ ]:
len(stations)

In [ ]:
from sklearn import manifold

# 非計量多次元尺度法を使う場合にはmetric=Falseを指定するが、残念ながらなぜかそれではちゃんと動かない。
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", max_iter=1000, n_init=100, metric=True)
pos = mds.fit_transform(table)


In [ ]:
pos

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties, FontManager

plt.scatter(pos[:,0], pos[:,1])

#fm = FontManager()
#print(fm.findfont('MS Gothic'))
#fp = FontProperties(fm.findfont('MS Gothic'), size=14)
#for label, x, y in zip(stations, pos[:, 0], pos[:, 1]):
#    plt.annotate(
#        label,
#        xy = (x, y), xytext = (60, 10), fontproperties=fp,
#        textcoords = 'offset points', ha = 'right', va = 'bottom',
#        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
#        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0')
#    )

大阪の人にとっては見慣れた地下鉄路線図のような形が表示された。残念ながら、matplotlibでの日本語表示がうまくいかないため、駅名との対応がよくわからない。

路線図の形から、料金表が現在のものではなく、けっこう古い情報であることが推測できる。

![現在の路線図](http://nansan.com/modules/images/download/rosenzu700.jpg)

## これは何に使えるだろうか

多次元尺度法は、一見何の関連性があるのかわからない多数の情報から、その構造を見付けだすのに使える。例えば、何かてきとうな尺度をx,y軸にとって情報をプロットすれば分布を描くことはできるが、いろんな情報が重なりあってしまうかもしれない。多次元尺度法により、適切な埋め込み空間の次元を見付けだせれば、誤って低次元でプロットしてしまい、異なる情報を重ねてしまうのを避けられる。

逆に、非常に複雑なからみあいを持つ情報を、(多少の歪みに目をつぶり)2〜3次元に埋め込むことで、高次元の情報を可視化することができる。

細胞の中や炎の中で起こる化学反応ネットワークは非常に複雑な構造を持つ。

分子集合体の構造変化の経路をつないだネットワークも非常に複雑な構造を持つ。

全体の構造を知らずに、個々の反応だけを見ていたのでは、大局的な理解ができない。

## 宿題
大江戸線の停車駅の駅間のへだたりの表から、大江戸線の路線図を推定せよ。可能なら図の上に駅名を示せ。(ネットでさがす)

In [ ]:
table = np.array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,
         2.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
        12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,
         3.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
       [ 2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,
        11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
         4.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
       [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,
        10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,
         5.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.],
       [ 4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,
         9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,
         6.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.],
       [ 5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,
         8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,
         7.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.],
       [ 6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,
         7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,
         8.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.],
       [ 7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,
         6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,
         9.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.],
       [ 8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,
         5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11.,
        10.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [ 9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,
         4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12.,
        11., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.],
       [10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,
         3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13.,
        12., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21.],
       [11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,
         2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        13., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.],
       [12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,
         1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.],
       [13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,
         0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.],
       [14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,
         1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
        12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.],
       [13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,
         2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,
        11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.],
       [12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,
         3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,
        10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21.],
       [11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
         4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,
         9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.],
       [10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,
         5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,
         8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [ 9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,  7.,
         6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,
         7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.],
       [ 8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,  8.,
         7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,
         6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.],
       [ 7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,  9.,
         8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,
         5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.],
       [ 6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11., 10.,
         9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,
         4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.],
       [ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12., 11.,
        10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,
         3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.],
       [ 4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13., 12.,
        11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,
         2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.],
       [ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 13.,
        12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,
         1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
       [ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,
         0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,
         1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [ 2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
        15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,
         2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [ 3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
        16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,
         3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.,
        17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
         4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,
         5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
       [ 6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.,
        19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,
         6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.],
       [ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.,
        20., 19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,
         7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.,
        21., 20., 19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,
         8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.],
       [ 9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21.,
        22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,
         9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.,
        23., 22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12., 11.,
        10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.],
       [11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.,
        24., 23., 22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12.,
        11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.]])

stations=["Shinjuku-nishiguchi",
"Higashi-shinjuku",
"Wakamatsu-kawada",
"Ushigome-yanagicho",
"Ushigome-kagurazaka",
"Iidabashi",
"Kasuga",
"Hongo-sanchome",
"Ueno-okachimachi",
"Shin-okachimachi",
"Kuramae",
"Ryogoku",
"Morishita",
"Kiyosumi-shirakawa",
"Monzen-nakacho",
"Tsukishima",
"Kachidoki",
"Tsukijishijo",
"Shiodome",
"Daimon",
"Akabanebashi",
"Azabu-juban",
"Roppongi",
"Aoyama-itchome",
"Kokuritsu-kyogijo",
"Yoyogi",
"Shinjuku",
"Tochomae",
"Nishi-shinjuku-gochome",
"Nakano-sakaue",
"Higashi-nakano",
"Nakai",
"Ochiai-minami-nagasaki",
"Shin-egota",
"Nerima",
"Toshimaen",
"Nerima-kasugacho",
"Hikarigaoka"]
